In [5]:
# Libraries
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import requests
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os, re, struct, socket

import import_ipynb
import project_models


In [3]:
#GPU Checking
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 3060 Ti


Data Preprocessing

In [4]:
#IDS2017_Monday ==== No anomalies
IDS2017_data = 'data\TrafficLabelling\Monday-WorkingHours.pcap_ISCX.csv'
IDS2017_DF = pd.DataFrame(pd.read_csv(IDS2017_data))


IDS2017_features = IDS2017_DF.iloc[:, [6, 1, 3, 2, 4, 5, 7, 84]] # 85 Columns
# Summing the total length of forward and backward packets
IDS2017_features['Total Length'] = IDS2017_DF.iloc[:, 11] + IDS2017_DF.iloc[:, 12]
IDS2017_features.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\TrafficLabelling\\Monday-WorkingHours.pcap_ISCX.csv'

In [ ]:
## KDDcup Corrected File
KDD_fp = 'data\kddcup\kddcup_corrected.csv'
KDD_DF = pd.DataFrame(pd.read_csv(KDD_fp))
KDD_DF.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type
0,tcp,http,SF,215,45076,0,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,tcp,http,SF,162,4528,0,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,tcp,http,SF,236,1228,0,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,tcp,http,SF,233,2032,0,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,tcp,http,SF,239,486,0,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [ ]:
## PCAP DATA
pcap_fp = 'data\pcap\midnight.csv'
pcap_DF = pd.DataFrame(pd.read_csv(pcap_fp))
pcap_DF.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,23.63.110.66,39.62.1.38,SSL,1498,Continuation Data
1,2,0.000000,23.63.110.66,39.62.1.38,TCP,1494,[TCP Retransmission] 443 > 37488 [ACK] Seq=1...
2,3,0.000013,103.12.198.146,14.2.1.224,SSL,1446,Continuation Data
3,4,0.000015,103.12.198.146,14.2.1.224,TCP,1446,[TCP Retransmission] 443 > 44502 [ACK] Seq=1...
4,5,0.000016,103.12.198.146,14.2.1.224,TCP,1454,[TCP Retransmission] 443 > 44502 [ACK] Seq=1...


In [ ]:
# Helper function to convert IP addresses into integers
def ip_to_int(ip_address):
    try:
        return struct.unpack("!L", socket.inet_aton(ip_address))[0]
    except socket.error:
        # Handle the case for invalid IP addresses
        return 0


#Info Extracter for Pcap type files
def extract_network_info(info_str):
    # Dictionary to hold the extracted values
    extracted_info = {
        'sending_port': None,
        'receiving_port': None,
        'length': None,
        'seq': None,
        'ack': None,
        'win': None,
        'tsval': None,
        'tsecr': None,
        'sle_sre': [],
        'info_messages': [],
    }
    
    # Check for and extract port and length information
    match = re.search(r'(\S+)\s+\d+\s+(\d+)\s*>\s*(\d+)(?:.*Len=(\d+))?', entry)
    if match:
        # Protocol and port information
        extracted_info['protocol'] = match.group(1)
        extracted_info['sending_port'] = int(match.group(2))
        extracted_info['receiving_port'] = int(match.group(3))
        
        # Length information if present
        if match.group(4):
            extracted_info['length'] = int(match.group(4))

    # Extract other information if present
    for key in ['seq', 'ack', 'win', 'tsval', 'tsecr']:
        match = re.search(r'{}=(\d+)'.format(key.capitalize()), info_str)
        if match:
            extracted_info[key] = int(match.group(1))

    # Extract SLE and SRE pairs
    extracted_info['sle_sre'] = re.findall(r'SLE=(\d+)\s*SRE=(\d+)', info_str)
    
    # Capture informational messages like "Ignored Unknown Record" or "Continuation Data"
    info_msg_match = re.search(r'\[.*?\]', info_str)
    if info_msg_match:
        extracted_info['info_messages'].append(info_msg_match.group(0))

    return extracted_info





Custom Dataset Construction

In [ ]:
## The Dataset
class NetworkDataset(Dataset):
    def __init__(self, directory_path):
        # Assuming CSV files are named in a sequential manner or have identifiable patterns
        self.files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')]
        self.files.sort()  # Ensure the order is correct

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = pd.read_csv(self.files[idx])
        info = data['Info'].apply(extract_network_info)
        
         # Encode source and destination IPs
        data['source_encoded'] = data['Source'].apply(lambda ip: struct.unpack("!L", socket.inet_aton(ip))[0])
        data['destination_encoded'] = data['Destination'].apply(lambda ip: struct.unpack("!L", socket.inet_aton(ip))[0])


        features = torch.tensor(data[['Time', 'source_encoded', 'destination_encoded', 'Protocol', 'Length']].values, dtype=torch.float32)

        ## Regression targets
        # Calculating bandwidth and throughput
        total_bytes = data['Length'].sum()  # Sum of all packet lengths
        total_time = data['Time'].max() - data['Time'].min()  # Duration of capture in seconds

        bandwidth = total_bytes / total_time if total_time > 0 else 0
        throughput = bandwidth  # Simplified

        latency = torch.tensor(data['latency'].values, dtype=torch.float32) # milliseconds usually
        # network_flow = Client -> Port -> Switch -> Master Swtich -> Firewall -> Inline Network Encryter -> Router -> Outbound                 
        # resource_usage = no RAM/CPU data available 


        ## Classification targets
        traffic_type = torch.tensor(pd.factorize(data['traffic_type'])[0], dtype=torch.long)
        anomaly_detection = torch.tensor(data['anomaly_label'].values, dtype=torch.long)
        network_resource_status = torch.tensor(pd.factorize(data['resource_status'])[0], dtype=torch.long)
        bottlenecks = torch.tensor(pd.factorize(data['bottleneck'])[0], dtype=torch.long)


        ## Ports
        # Parsing port numbers from the Info column
        data['ports'] = data['Info'].apply(lambda x: re.findall(r'\d+', x))
       
        # Combine ports into a single feature
        data['combined_ports'] = data['ports'].apply(lambda x: f"{x[0]}-{x[1]}" if len(x) > 1 else None)
        #Factorize Ports
        combined_ports = torch.tensor(pd.factorize(data['combined_ports'])[0], dtype=torch.int64)
        
        
        
        return {
        'info': info,
        'features': features,
        'bandwidth': bandwidth,
        'throughput': throughput,
        'traffic_type': traffic_type,
        'anomaly_detection': anomaly_detection,
        'network_resource_status': network_resource_status,
        'bottlenecks': bottlenecks
    }
    

## Using the Dataset
dataset = NetworkDataset('data')
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# BATCH SIZE MUST BE 1 --> EACH CSV IS ITS OWN BATCH
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

'\nbatch_size = 1 # BATCH SIZE MUST BE 1 --> EACH CSV IS ITS OWN BATCH\ntrain_size = int(len(dataset) * 0.8)\ntest_size = len(dataset) - train_size\ntrain_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])\n\ntrain_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)\ntest_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)\n'

In [ ]:
input_size = 
hidden_size = 128
output_size = 1
dropout = 0.2
bidirectional = False

model = project_models.ntwkRNN(input_size, hidden_size, output_size, dropout, bidirectional)

epochs = 10
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters() lr = 0.01)

rnn_tloss, rnn_vloss,rnn_vacc = project_models.train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device)

In [ ]:
model = project_models.ntwkGRU(input_size, hidden_size, output_size, dropout, bidirectional)
optimizer = optim.Adam(model.parameters() lr = 0.01)

GRU_tloss, GRU_vloss,GRU_vacc = project_models.train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device)

In [ ]:
model = project_models.ntwkLSTM(input_size, hidden_size, output_size, dropout, bidirectional)
optimizer = optim.Adam(model.parameters() lr = 0.01)

GRU_tloss, GRU_vloss, GRU_vacc = project_models.train_RNN(epochs, model, criterion, optimizer, train_loader, test_loader, device)

In [ ]:
model = project_models.ntwkPETransformer(input_size, hidden_size, output_size, num_layers = 5, nhead = 3)
optimizer = optim.Adam(model.parameters() lr = 0.01)


Trans_tloss, Trans_vloss, Trans_vacc = project_models.train_Transformer(model,epochs, criterion, optimizer, train_loader, test_loader, device)